In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType, ArrayType, FloatType
from pyspark.sql.functions import expr, lower, col
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, Tokenizer, StopWordsRemover, CountVectorizer, StandardScaler
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
import mlflow
import mlflow.spark
import mlflow.pyfunc
from mlflow.models import infer_signature, ModelSignature
from mlflow.models.signature import ModelSignature
from mlflow.types import Schema, ColSpec
from mlflow.tracking import MlflowClient
from mlflow import MlflowClient

In [0]:
DATABRICKS_UC = 'databricks-uc'
DATABRICKS_BEST_MODEL_TAG =  "Champion"
TEST_METRIC = 'accuracy_score'
MODEL_NAME = 'perceptron_description_to_category'
MODEL_NAMES = f'tmp_cat_dev.aranda_ml.{MODEL_NAME}'
ERROR_MESSAGE = "El valor ingresado no es un número."
notebook_path = "/notebook/src/baseline.ipynb"
PARAMETER_VALUE = "false"
PARAMETER_NAME = "LOCAL_EVALUATION"

In [0]:
dbutils.widgets.text("METRIC_EVALUATION", "0", "Set METRIC_EVALUATION (0/number)")
METRIC_EVALUATION_INPUT = dbutils.widgets.get("METRIC_EVALUATION")
if METRIC_EVALUATION_INPUT.isnumeric():
    METRIC_EVALUATION = float(METRIC_EVALUATION_INPUT)
else:
    dbutils.notebook.exit(ERROR_MESSAGE)

In [0]:
client = MlflowClient()
mlflow.set_registry_uri(DATABRICKS_UC)
model_version = client.get_model_version_by_alias(MODEL_NAMES, DATABRICKS_BEST_MODEL_TAG)
run = client.get_run(model_version.run_id)
metric = run.data.metrics.get(TEST_METRIC)
if METRIC_EVALUATION > float(metric):
    try:
        dbutils.notebook.run(notebook_path, 600, {PARAMETER_NAME: PARAMETER_VALUE})
        dbutils.notebook.exit(str(1))
    except Exception as e:
        pass
dbutils.notebook.exit(str(0))